In [1]:
from bs4 import BeautifulSoup
import requests
import ast
import pandas as pd

# Alta Web Scraping

In [61]:
cntry_link = "https://www.alta.ru/railway/01/"
soup = BeautifulSoup(requests.get(cntry_link).text)
div = soup.find("div", {"class": "pRailway_body mFastSearch"})
script = div.find_all("script")[1]
script

In [64]:
coords = script.text[script.text.find("coords"):script.text.find("var cc")]
coords = coords.replace("\n", "").replace("\t", "")[9:-1]
coords = ast.literal_eval(coords)
print(len(coords))

642


In [6]:
names = script.text[script.text.find("var cc")+9:script.text.find("var bcb")].replace("\n", "").replace("\t", "")[:-1]
names = ast.literal_eval(names)
print(len(names))

291


In [7]:
desc = script.text[script.text.find("var bcb")+10:script.text.find("var myGeoObjects")].replace("\n", "").replace("\t", "")[:-1]
desc = ast.literal_eval(desc)
print(len(desc))

291


In [8]:
coords = pd.DataFrame(coords)
names = pd.DataFrame(names)
desc = pd.DataFrame(desc)

df = pd.concat([names, coords, desc], axis=1)
df.columns = ["name", "lat", "lon", "description"]
df['no'] = df["name"].str.split().str[0]
df

,name,lat,lon,description,no
0,40800 Абамеликово,47.979025,29.262881,<span>40800 Станция Абамеликово</span> <br>,40800
1,42380 Адабаш,48.353194,31.479798,<span>42380 Станция Адабаш</span> <br> (3) При...,42380
2,40190 Аккаржа,46.331985,30.559101,<span>40190 Станция Аккаржа</span> <br> (3) Пр...,40190
3,41040 Александрия,48.674014,33.133147,<span>41040 Станция Александрия</span> <br> (3...,41040
4,41310 Александровка,47.641216,31.300756,<span>41310 Станция Александровка</span> <br> ...,41310
...,...,...,...,...,...
286,42200 Шпола,48.995573,31.399416,<span>42200 Станция Шпола</span> <br> (3) Прие...,42200
287,41330 Южноукраинская,47.764449,31.277198,<span>41330 Станция Южноукраинская</span> <br>...,41330
288,41490 Явкино,47.319356,32.469439,<span>41490 Станция Явкино</span> <br> (3) При...,41490
289,41073 Яковлевка,48.450291,33.619122,<span>41073 Станция Яковлевка</span> <br>,41073


# Final Loop

In [102]:
def get_stations(cntry_link):
    soup = BeautifulSoup(requests.get(cntry_link).text)
    div = soup.find("div", {"class": "pRailway_body mFastSearch"})
    script = div.find_all("script")[1]
    
    coords = script.text[script.text.find("coords"):script.text.find("var cc")]
    coords = coords.replace("\n", "").replace("\t", "")[9:-1]
    coords = coords.replace('[,', '[')
    coords = ast.literal_eval(coords)

    names = script.text[script.text.find("var cc")+9:script.text.find("var bcb")].replace("\n", "").replace("\t", "")[:-1]
    names = ast.literal_eval(names)

    assert len(names) == len(coords)
    
    desc = script.text[script.text.find("var bcb")+10:script.text.find("var myGeoObjects")].replace("\n", "").replace("\t", "")[:-1]
    desc = ast.literal_eval(desc)

    assert len(names) == len(desc)
    
    coords = pd.DataFrame(coords)
    names = pd.DataFrame(names)
    desc = pd.DataFrame(desc)
    
    df = pd.concat([names, coords, desc], axis=1)
    df.columns = ["name", "lat", "lon", "description"]
    df['no'] = df["name"].str.split().str[0]

    return df


In [103]:
RU_RAILROADS_ID = ['01', 10, 17, 24, 28, 32, 51, 58, 63, 68, 76, 80, 83, 88, 91, 92, 94, 96]

In [104]:
ru_roads = list(map("https://www.alta.ru/railway/{}/".format, RU_RAILROADS_ID))

ru_roads

['https://www.alta.ru/railway/01/',
 'https://www.alta.ru/railway/10/',
 'https://www.alta.ru/railway/17/',
 'https://www.alta.ru/railway/24/',
 'https://www.alta.ru/railway/28/',
 'https://www.alta.ru/railway/32/',
 'https://www.alta.ru/railway/51/',
 'https://www.alta.ru/railway/58/',
 'https://www.alta.ru/railway/63/',
 'https://www.alta.ru/railway/68/',
 'https://www.alta.ru/railway/76/',
 'https://www.alta.ru/railway/80/',
 'https://www.alta.ru/railway/83/',
 'https://www.alta.ru/railway/88/',
 'https://www.alta.ru/railway/91/',
 'https://www.alta.ru/railway/92/',
 'https://www.alta.ru/railway/94/',
 'https://www.alta.ru/railway/96/']

In [106]:
ru_stations = pd.DataFrame()
for link in ru_roads:
    ru_stations = pd.concat([ru_stations, get_stations(link)], axis=0)
ru_stations.to_csv("ru_stations.csv", encoding='utf-8')

In [110]:
ru_stations.columns = ['name_ru', 'lat', 'lon', 'd', 'esr']
ru_stations = ru_stations.drop(columns=['d'])
ru_stations['source'] = 'alta'

In [112]:
df = pd.read_excel("../stations.xlsx", index_col=0)
df = pd.concat([df, ru_stations], axis=0)
df = df.reset_index(drop=True)
df.to_excel('../stations.xlsx')

,esr,name_ru,name_ukr,name_en,lat,lon,year_built,year_built_is_null,source
0,320685,ОП Вышгород,NaN,NaN,50.584732,30.510171,NaN,1.0,manual
1,324807,Жукотки,NaN,NaN,50.276565,30.938461,NaN,1.0,manual
2,324926,ОП Ковпыта,NaN,NaN,50.272735,30.763733,NaN,1.0,manual
3,336013,ОП Вергиевка,NaN,NaN,50.302530,30.682672,NaN,1.0,manual
4,340807,Сельцо,NaN,NaN,50.455534,30.663700,NaN,1.0,manual
...,...,...,...,...,...,...,...,...,...
187,43310,43310 Шиповатое,NaN,NaN,49.941769,37.440770,NaN,NaN,alta
188,44460,44460 Шпаковка,NaN,NaN,49.994042,36.002940,NaN,NaN,alta
189,43090,43090 Эсхар,NaN,NaN,49.827488,36.562873,NaN,NaN,alta
190,42660,42660 Юсковцы,NaN,NaN,50.469441,33.330236,NaN,NaN,alta
